## The Problem

In [1]:
import pandas as pd
import psycopg2

# Load all records from holdings_new where quarter_end = '2025-06-30'
query_all_holdings = """
SELECT *
FROM holdings_new
WHERE quarter_end = '2025-06-30'
"""

conn = psycopg2.connect(
    dbname="Social_13F",
    user="postgres",
    password="Test123",
    host="127.0.0.1",
    port=5432
)

df_holdings_jun2025 = pd.read_sql_query(query_all_holdings, conn)
conn.close()

df_holdings_jun2025.drop(columns=['accessionnumber','infotablesk'], inplace=True)

# Convert all string columns in the dataframe to lowercase
# Convert only columns that are string/object dtype to string first, then lowercase safely
for col in df_holdings_jun2025.select_dtypes(include="object").columns:
    df_holdings_jun2025[col] = df_holdings_jun2025[col].astype(str).str.lower()

df_holdings_jun2025.head()


C:\Users\potda\AppData\Local\Temp\ipykernel_25424\2736075059.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_holdings_jun2025 = pd.read_sql_query(query_all_holdings, conn)


,nameofissuer,cusip,value,sshprnamt,filingdate,cik,quarter_end,value_per_share
0,3m co,88579y101,7308.0,48.0,2025-08-29,0002010029,2025-06-30,152.240000
1,abbott labs,002824100,90855.0,668.0,2025-08-29,0002010029,2025-06-30,136.009999
2,abbvie inc,00287y109,25987.0,140.0,2025-08-29,0002010029,2025-06-30,185.620000
3,abrdn healthcare opportuniti,879105104,14822.0,806.0,2025-08-29,0002010029,2025-06-30,18.390000
4,adient plc,g0084w101,39.0,2.0,2025-08-29,0002010029,2025-06-30,19.460000


In [2]:
df_holdings_jun2025.shape

(2989722, 8)

number of unique cusip in Q2_2025

In [3]:
len(df_holdings_jun2025['cusip'].unique())

22244

lets see how much (nameofissuer,cusip) does APPLE have

In [4]:
query_apple = """
SELECT cusip,nameofissuer, COUNT(*) as record_count
FROM holdings_new
WHERE nameofissuer LIKE '%APPLE%'
AND quarter_end = '2025-06-30'
GROUP BY cusip,nameofissuer
ORDER BY COUNT(*) DESC;
"""

df_apple = pd.read_sql_query(query_apple, psycopg2.connect(
    dbname="Social_13F",
    user="postgres",
    password="Test123",
    host="127.0.0.1",
    port=5432
))
df_apple


C:\Users\potda\AppData\Local\Temp\ipykernel_25424\1228503417.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_apple = pd.read_sql_query(query_apple, psycopg2.connect(


,cusip,nameofissuer,record_count
0,037833100,APPLE INC,7210
1,03784Y200,APPLE HOSPITALITY REIT INC,419
2,037833100,APPLE INC COM,342
3,037833100,APPLE,76
4,577345101,MAUI LD & PINEAPPLE INC,70
...,...,...,...
63,577345101,MAUI LD&PINEAPPLE INC COM,1
64,65370G109,NICHOLAS APPLEGATE,1
65,71377A103,APPLE HOSPITALITY REIT INC,1
66,78433H725,KURV YIELD PREMIUM STRATEGY APPLE (AAPL) ETF,1


lets see decsending unique(cusip) by groupby

In [5]:
query_apple = """
SELECT cusip, COUNT(*) as record_count
FROM holdings_new
WHERE nameofissuer LIKE '%APPLE%'
AND quarter_end = '2025-06-30'
GROUP BY cusip
ORDER BY COUNT(*) DESC;
"""

df_apple = pd.read_sql_query(query_apple, psycopg2.connect(
    dbname="Social_13F",
    user="postgres",
    password="Test123",
    host="127.0.0.1",
    port=5432
))
df_apple


C:\Users\potda\AppData\Local\Temp\ipykernel_25424\599844435.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_apple = pd.read_sql_query(query_apple, psycopg2.connect(


,cusip,record_count
0,037833100,7766
1,03784Y200,474
2,577345101,93
3,037833950,4
4,72303K207,4
5,037833900,3
6,H50430232,1
7,037833BY5,1
8,65370G109,1
9,71377A103,1


## Import the russell3000 index and find for each ticker his unique cusip

load the russell3000 index symbols from the file and produce 3word by name

In [6]:
import pandas as pd

# Load the CSV
russell3000_df = pd.read_csv("..\Data\Indexes\Holdings_details_Russell_3000_ETF.csv", header=None, names=['HOLDINGS', 'TICKER', 'SECTOR'])

# Remove the first row if it's a repeated header (shouldn't appear in final DataFrame)
if (russell3000_df.iloc[0] == ['holdings', 'ticker', 'sector']).all():
    russell3000_df = russell3000_df.iloc[1:].reset_index(drop=True)

# Lowercase the entire DataFrame (string columns only)
russell3000_df = russell3000_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

# Function to get first 3 words from the HOLDINGS column
def first_three_words(name):
    words = str(name).split()
    return " ".join(words[:3])

# Add new column with first 3 words from the HOLDINGS column
russell3000_df["HOLDINGS_3WORDS"] = russell3000_df["HOLDINGS"].apply(first_three_words)

# Remove again any potential accidental row where HOLDINGS_3WORDS == 'holdings', to ensure header isn't present
russell3000_df = russell3000_df[russell3000_df["HOLDINGS_3WORDS"] != "holdings"].reset_index(drop=True)

# Display (optional)
russell3000_df


C:\Users\potda\AppData\Local\Temp\ipykernel_25424\1364010025.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  russell3000_df = russell3000_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


,HOLDINGS,TICKER,SECTOR,HOLDINGS_3WORDS
0,nvidia corp,nvda,semiconductors,nvidia corp
1,apple inc,aapl,"technology hardware, storage & peripherals",apple inc
2,microsoft corp,msft,systems software,microsoft corp
3,amazon.com inc,amzn,broadline retail,amazon.com inc
4,broadcom inc,avgo,semiconductors,broadcom inc
...,...,...,...,...
2945,gohealth inc,goco,insurance brokers,gohealth inc
2946,gemini space station inc,gemi,financial exchanges & data,gemini space station
2947,third harmonic bio inc,thrd,pharmaceuticals,third harmonic bio
2948,gci liberty inc-cl a,gliba,integrated telecommunication services,gci liberty inc-cl


find cusip to ticker by the most frequent cusip for 3-->words

In [7]:
# 1. Pre-filter the relevant quarter once to avoid repeated filtering inside the loop
holdings_q = df_holdings_jun2025[df_holdings_jun2025['quarter_end'] == "2025-06-30"].copy()

# 2. Optimization Stage: Map unique issuers to their most frequent CUSIP
# This drastically reduces the search space from millions of records to a few thousand unique names.
issuer_to_top_cusip = (
    holdings_q.groupby(['nameofissuer', 'cusip'])
    .size()
    .reset_index(name='count')
    .sort_values(['nameofissuer', 'count'], ascending=[True, False])
    .drop_duplicates('nameofissuer')
)

# Store unique issuer names and their top CUSIPs in arrays for faster iteration
unique_issuers = issuer_to_top_cusip['nameofissuer'].values
top_cusips = issuer_to_top_cusip['cusip'].values

cusip_results = []

# 3. Main Matching Loop
# Instead of searching the full DataFrame, we search only within the unique issuer list
for idx, row in russell3000_df.iterrows():
    threewords = row['HOLDINGS_3WORDS']
    
    # 1. Create a boolean mask for all unique issuers that contain the 3 words
    mask = [threewords in issuer for issuer in unique_issuers]
    
    if any(mask):
        # 2. Get all matching rows from our aggregated table
        # Since 'mask' corresponds to the rows in 'issuer_to_top_cusip'
        matches = issuer_to_top_cusip[mask]
        
        # 3. Find the row with the maximum global count among these matches
        # This mirrors the 'ORDER BY count(*) DESC LIMIT 1' logic from your SQL
        best_row_index = matches['count'].idxmax()
        best_cusip = matches.loc[best_row_index, 'cusip']
        
        cusip_results.append(best_cusip)
    else:
        cusip_results.append(None)

# 4. Update the original DataFrame with the optimized results
russell3000_df["CUSIP"] = cusip_results


check this method by SPY banchmark to estimate the accuracy of the system

In [8]:
import pandas as pd

def load_and_lowercase_spy(filepath):
    """Load SPY CSV and lowercase all string columns."""
    spy_df = pd.read_csv(filepath)
    for col in spy_df.columns:
        if spy_df[col].dtype == 'O':
            spy_df[col] = spy_df[col].str.lower()
    return spy_df

def lowercase_df_columns(df):
    """Return a copy of df with all string columns lowercased."""
    new_df = df.copy()
    for col in new_df.columns:
        if new_df[col].dtype == 'O':
            new_df[col] = new_df[col].str.lower()
    return new_df

def merge_by_ticker(russell3000_df, spy_df):
    """Merge russell3000 and SPY dataframes on ticker columns."""
    merged = pd.merge(
        russell3000_df,
        spy_df,
        left_on='TICKER',
        right_on='Ticker',
        suffixes=('', '_SPY'),
        how='outer',
        indicator=True
    )
    return merged

def build_comparison_df(merged_df):
    """Create a DataFrame comparing found vs. ground-truth CUSIPs."""
    # Use appropriate column for truth
    if 'CUSIP_SPY' in merged_df.columns:
        cusip_truth = merged_df['CUSIP_SPY']
    else:
        cusip_truth = merged_df['CUSIP']
    result_compare = pd.DataFrame({
        'TICKER': merged_df['TICKER'].combine_first(merged_df['Ticker']),
        'cusip_found': merged_df['CUSIP'],
        'cusip_truth': cusip_truth,
        'source': merged_df['_merge']
    })
    result_compare['is_match'] = result_compare['cusip_found'] == result_compare['cusip_truth']
    return result_compare

def cusip_match_stats(compare_df, verbose=True):
    """Print match stats and show matched/unmatched DataFrames."""
    matched = compare_df[compare_df['source'] == 'both']
    num_matches = matched['is_match'].sum()
    total_compared = len(matched)
    percent_matches = (num_matches / total_compared) * 100 if total_compared else 0

    if verbose:
        print(f"Matched CUSIPs: {num_matches} / {total_compared} ({percent_matches:.2f}%)")

    matched_df = matched[matched['is_match']]
    unmatched_df = matched[~matched['is_match']]
    if verbose:
        print("First matches:")
        print(matched_df.head())
        print("\nFirst non-matches:")
        print(unmatched_df.head())
    return matched_df, unmatched_df, percent_matches

# ==== PIPELINE ====
# 1. Load and clean SPY dataframe
spy_df = load_and_lowercase_spy("../Data/Indexes/SPY.csv")

# 2. Lowercase all columns of russell3000 (copy)
russell3000_lower = lowercase_df_columns(russell3000_df)

# 3. Merge on TICKER
merged = merge_by_ticker(russell3000_lower, spy_df)

# 4. Build comparison table
result_compare = build_comparison_df(merged)

# 5. Analyze matches and print results
matched_df, unmatched_df, percent_matches = cusip_match_stats(result_compare, verbose=True)


Matched CUSIPs: 490 / 496 (98.79%)
First matches:
   TICKER cusip_found cusip_truth source  is_match
1       a   00846u101   00846u101   both      True
8    aapl   037833100   037833100   both      True
12   abbv   00287y109   00287y109   both      True
18   abnb   009066101   009066101   both      True
21    abt   002824100   002824100   both      True

First non-matches:
     TICKER cusip_found cusip_truth source  is_match
652     cpb        None   134429109   both     False
1067    fox   35137l105   35137l204   both     False
1185   goog   02079k305   02079k107   both     False
1923    nws   65249b109   65249b208   both     False
2188      q        None   74743l100   both     False


In [9]:
tickersunmatched = unmatched_df['TICKER'].tolist()
russell3000_df[russell3000_df['TICKER'].isin(tickersunmatched)]

,HOLDINGS,TICKER,SECTOR,HOLDINGS_3WORDS,CUSIP
6,alphabet inc,goog,interactive media & services,alphabet inc,02079k305
195,slb ltd,slb,oil & gas equipment & services,slb ltd,None
463,qnity electronics inc,q,semiconductor materials & equipment,qnity electronics inc,None
703,fox corp,fox,broadcasting,fox corp,35137l105
861,campbell's company/the,cpb,packaged foods & meats,campbell's company/the,None
1583,news corp,nws,publishing,news corp,65249b109


In [10]:
russell3000_df.drop(
    russell3000_df[russell3000_df['CUSIP'].isna() | (russell3000_df['CUSIP'].str.strip() == '')].index,
    inplace=True
)

In [11]:
# Get a list of unique CUSIP values left in russell3000_df
cusip_list = russell3000_df['CUSIP'].unique().tolist()

# Filter df_holdings_jun2025 to keep only records where the 'cusip' is in cusip_list
before_holdings = len(df_holdings_jun2025)
c_before, r_before = df_holdings_jun2025['cusip'].nunique(), len(df_holdings_jun2025)

df_holdings_jun2025 = df_holdings_jun2025[df_holdings_jun2025['cusip'].isin(cusip_list)]
after_holdings = len(df_holdings_jun2025)

print(f"Rows remaining in df_holdings_jun2025 after filtering: {after_holdings}")
print(f"Rows dropped from df_holdings_jun2025: {before_holdings - after_holdings}")


# filtering already done above
c_after, r_after = df_holdings_jun2025['cusip'].nunique(), len(df_holdings_jun2025)
print(f"CUSIP before: {c_before}, after: {c_after} (diff: {c_before-c_after}, {100*(c_before-c_after)/c_before:.2f}%)")
print(f"Rows before: {r_before}, after: {r_after} (diff: {r_before-r_after}, {100*(r_before-r_after)/r_before:.2f}%)")


Rows remaining in df_holdings_jun2025 after filtering: 1966154
Rows dropped from df_holdings_jun2025: 1023568
CUSIP before: 22244, after: 2866 (diff: 19378, 87.12%)
Rows before: 2989722, after: 1966154 (diff: 1023568, 34.24%)


## load Index Tickers --> CUSIP

## get historyprices for thr tickers for all qaurters

In [ ]:
EDOHD_API="key in env"

In [13]:
import psycopg2

# Connect to the database (update with your credentials as needed)
conn = psycopg2.connect(
    dbname="Social_13F",
    user="postgres",
    password="Test123",
    host="127.0.0.1",
    port="5432"
)

# Query all unique dates from quarter_end column in holdings_new table
query_unique_quarters = "SELECT DISTINCT quarter_end FROM holdings_new ORDER BY quarter_end;"

with conn.cursor() as cur:
    cur.execute(query_unique_quarters)
    # Save the unique dates to a list
    unique_quarter_dates = [row[0] for row in cur.fetchall()]

print(unique_quarter_dates)
conn.close()



[datetime.date(2013, 6, 30), datetime.date(2013, 9, 30), datetime.date(2013, 12, 31), datetime.date(2014, 3, 31), datetime.date(2014, 6, 30), datetime.date(2014, 9, 30), datetime.date(2014, 12, 31), datetime.date(2015, 3, 31), datetime.date(2015, 6, 30), datetime.date(2015, 9, 30), datetime.date(2015, 12, 31), datetime.date(2016, 3, 31), datetime.date(2016, 6, 30), datetime.date(2016, 9, 30), datetime.date(2016, 12, 31), datetime.date(2017, 3, 31), datetime.date(2017, 6, 30), datetime.date(2017, 9, 30), datetime.date(2017, 12, 31), datetime.date(2018, 3, 31), datetime.date(2018, 6, 30), datetime.date(2018, 9, 30), datetime.date(2018, 12, 31), datetime.date(2019, 3, 31), datetime.date(2019, 6, 30), datetime.date(2019, 9, 30), datetime.date(2019, 12, 31), datetime.date(2020, 3, 31), datetime.date(2020, 6, 30), datetime.date(2020, 9, 30), datetime.date(2020, 12, 31), datetime.date(2021, 3, 31), datetime.date(2021, 6, 30), datetime.date(2021, 9, 30), datetime.date(2021, 12, 31), datetime.d

In [14]:
pip install pandas_market_calendars --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
test_dates = unique_quarter_dates.tolist() if hasattr(unique_quarter_dates, 'tolist') else list(unique_quarter_dates)
# Convert dates to string format 'YYYY-MM-DD' for API compatibility
test_dates = [d.strftime('%Y-%m-%d') if hasattr(d, 'strftime') else str(d) for d in test_dates]


In [16]:
import pandas_market_calendars as mcal
import pandas as pd

# Convert test_dates to pandas Timestamp
test_dates_pd = [pd.Timestamp(d) for d in test_dates]

# Get NYSE trading calendar
nyse = mcal.get_calendar('NYSE')

# Determine the date range needed from test_dates
min_date = min(test_dates_pd)
max_date = max(test_dates_pd)

# Query trading days for the range of dates in test_dates
calendar_schedule = nyse.schedule(start_date=min_date, end_date=max_date)
trading_days = set(calendar_schedule.index)

# For each date, if not trading day, move to next valid trading day
adjusted_dates = []
for date in test_dates_pd:
    if date in trading_days:
        adjusted_dates.append(date)
        print(f"{date.strftime('%Y-%m-%d')} is a trading day: True")
    else:
        # Find next trading day after this date
        future_trading_days = [d for d in trading_days if d > date]
        if future_trading_days:
            next_trading_day = min(future_trading_days)
            adjusted_dates.append(next_trading_day)
            print(f"{date.strftime('%Y-%m-%d')} is not a trading day. Set to: {next_trading_day.strftime('%Y-%m-%d')}")
        else:
            adjusted_dates.append(date)
            print(f"{date.strftime('%Y-%m-%d')} is not a trading day, and no future trading day found.")

# Update test_dates to be the adjusted trading days list in string format
test_dates = [d.strftime('%Y-%m-%d') for d in adjusted_dates]


2013-06-30 is not a trading day. Set to: 2013-07-01
2013-09-30 is a trading day: True
2013-12-31 is a trading day: True
2014-03-31 is a trading day: True
2014-06-30 is a trading day: True
2014-09-30 is a trading day: True
2014-12-31 is a trading day: True
2015-03-31 is a trading day: True
2015-06-30 is a trading day: True
2015-09-30 is a trading day: True
2015-12-31 is a trading day: True
2016-03-31 is a trading day: True
2016-06-30 is a trading day: True
2016-09-30 is a trading day: True
2016-12-31 is not a trading day. Set to: 2017-01-03
2017-03-31 is a trading day: True
2017-06-30 is a trading day: True
2017-09-30 is not a trading day. Set to: 2017-10-02
2017-12-31 is not a trading day. Set to: 2018-01-02
2018-03-31 is not a trading day. Set to: 2018-04-02
2018-06-30 is not a trading day. Set to: 2018-07-02
2018-09-30 is not a trading day. Set to: 2018-10-01
2018-12-31 is a trading day: True
2019-03-31 is not a trading day. Set to: 2019-04-01
2019-06-30 is not a trading day. Set to:

In [17]:

import requests
import time
# Helper function to fetch adjusted close price for a single ticker on a single date from EODHD API
def fetch_adjusted_close(ticker, date, api_key, session=None, max_retries=3):
    """
    Fetch adjusted close price for given ticker and date using EODHD API.

    Parameters:
        ticker (str): The stock ticker.
        date (str): Date in 'YYYY-MM-DD' format.
        api_key (str): EODHD API key.
        session (requests.Session): An optional requests session.
        max_retries (int): Max retries for network errors.

    Returns:
        float or None: Adjusted close price, or None if not found.
    """
    # For Russell 3000, most are US stocks on "US" exchange
    url = f"https://eodhd.com/api/eod/{ticker}.US"
    params = {
        'api_token': api_key,
        'from': date,
        'to': date,
        'period': 'd',
        'fmt': 'json',
        'adjusted': 1,
    }
    session = session or requests.Session()
    for attempt in range(max_retries):
        try:
            response = session.get(url, params=params)
            if response.status_code == 200:
                data = response.json()
                if type(data) is list and len(data) > 0:
                    return data[0].get('adjusted_close')
                elif type(data) is dict and 'code' in data:
                    # API returned error, e.g. {'code': 'TickerNotFound'}
                    return None
                else:
                    return None
            elif response.status_code == 429:
                # Rate limit. Wait and retry.
                time.sleep(5)
            else:
                return None
        except Exception:
            time.sleep(1)
    return None

In [18]:
# Helper to map tickers to the correct EODHD format/append '.US'. Customize if needed for exceptions.
def get_eodhd_ticker(ticker):
    # Place for any cleaning/formatting per EODHD requirements
    return f"{ticker.upper()}"

In [19]:
# Function to build close price DataFrame for a single date for all tickers
def build_close_price_dataframe_for_date(tickers, date, api_key, rate_limit_per_min=900, max_workers=12):
    """
    Fetch adjusted close prices for list of tickers for specific date using parallel requests,
    while enforcing a rate limit of 900 requests per minute.

    Returns a DataFrame: one row per ticker, column = date, value = price (float or None if not available).

    Args:
        tickers (list): List of ticker strings.
        date (str): Date string ('YYYY-MM-DD').
        api_key (str): API key for EODHD.
        rate_limit_per_min (int): Max API calls per minute (default 900).
        max_workers (int): Number of parallel threads/processes (default 8).

    Returns:
        pd.DataFrame: Ticker and price info.
    """
    import pandas as pd
    import time
    from concurrent.futures import ThreadPoolExecutor, as_completed
    from threading import BoundedSemaphore

    # Set up a semaphore to allow up to rate_limit_per_min requests per minute
    sem = BoundedSemaphore(rate_limit_per_min)

    # Timestamps of requests to track per-minute limits
    request_times = []

    # Helper function to call API and apply rate limiting
    def fetch_for_ticker(ticker):
        nonlocal request_times
        session = requests.Session()
        mapped_ticker = get_eodhd_ticker(ticker)
        # Rate limiting logic:
        while True:
            now = time.time()
            # Remove timestamps older than 60 seconds
            request_times = [t for t in request_times if now - t < 60]
            if len(request_times) < rate_limit_per_min:
                request_times.append(now)
                break
            time_to_wait = 60 - (now - request_times[0])
            if time_to_wait > 0:
                time.sleep(min(time_to_wait, 0.05))
        price = fetch_adjusted_close(mapped_ticker, date, api_key, session)
        return {
            "TICKER": ticker,
            date: price
        }

    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_ticker = {executor.submit(fetch_for_ticker, ticker): ticker for ticker in tickers}
        completed = 0
        for future in as_completed(future_to_ticker):
            result = future.result()
            results.append(result)
            completed += 1
            print(f"Fetched {completed}/{len(tickers)}: {result['TICKER']} -> {result[date]}")

    df = pd.DataFrame(results)
    return df

In [ ]:
# Set your API key here
API_KEY = "in env!"  # Set safely/securely

# Select the first date from test_dates
# target_date = test_dates[0]  # e.g. '2025-06-30'
# print(f"Target date: {target_date}")

# # Get unique tickers from russell3000_df
# tickers_list = russell3000_df['TICKER'].str.upper().unique().tolist()
# print(f"Total tickers to fetch: {len(tickers_list)}")

# # Fetch prices for ALL tickers for the first date
# result_df = build_close_price_dataframe_for_date(tickers_list, target_date, API_KEY)

# # Display results
# print(f"\nFetched prices for {len(result_df)} tickers")
# print(result_df.head(20))

# # Show how many have valid prices vs None
# valid_prices = result_df[target_date].notna().sum()
# missing_prices = result_df[target_date].isna().sum()
# print(f"\nValid prices: {valid_prices}, Missing prices: {missing_prices}")

In [ ]:
# For each of the first 25 dates, fetch prices for all tickers and upsert results immediately into the PostgreSQL table 'tickerprices'.
# Each row will include: ticker, date, price, and quarter_end (the nearest quarter-end date to 'date').
# Also, collect per-date statistics and store summary in a DataFrame at the end.
# Use only psycopg2, since SQLAlchemy is NOT available.
# The code will check for a closed connection and reconnect if needed.
# All comments are in English.

from psycopg2.extras import execute_batch
import datetime

def map_to_nearest_quarter_end(date_str):
    """Map a string date (YYYY-MM-DD) to the closest quarter-end in the same year."""
    date = pd.to_datetime(date_str)
    quarter_ends = [
        datetime.date(date.year, 3, 31),
        datetime.date(date.year, 6, 30),
        datetime.date(date.year, 9, 30),
        datetime.date(date.year, 12, 31),
    ]
    closest = min(quarter_ends, key=lambda q: abs(date.date() - q))
    return str(closest)

def ensure_conn_open(conn):
    """Reconnects if the psycopg2 connection has closed."""
    try:
        # Try a simple operation to check connection
        if conn.closed != 0:
            # Reconnect using saved parameters, adjust as needed
            return psycopg2.connect(
                dbname="Social_13F",
                user="postgres",
                password="Test123",
                host="127.0.0.1",
                port=5432
            )
        else:
            return conn
    except Exception:
        # If any error, force reconnect
        return psycopg2.connect(
            dbname="Social_13F",
            user="postgres",
            password="Test123",
            host="127.0.0.1",
            port=5432
        )

stats = []
tickers_list = russell3000_df['TICKER'].str.upper().unique().tolist()

for idx, target_date in enumerate(test_dates[26:]):
    print(f"Fetching for {target_date} ...")

    # Fetch the price DataFrame for the current date
    date_prices_df = build_close_price_dataframe_for_date(tickers_list, target_date, API_KEY)
    date_prices_df["date"] = target_date
    date_prices_df = date_prices_df.rename(columns={target_date: "price"})
    date_prices_df = date_prices_df[["TICKER", "date", "price"]]

    # Compute quarter_end column for each entry
    date_prices_df["quarter_end"] = date_prices_df["date"].apply(map_to_nearest_quarter_end)

    # Prepare tuples for database insertion
    data_tuples = [tuple(x) for x in date_prices_df[["TICKER", "date", "price", "quarter_end"]].to_numpy()]

    # Compose insert statement
    insert_stmt = """
        INSERT INTO tickerprices (ticker, date, price, quarter_end)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (ticker, date) DO UPDATE SET price=EXCLUDED.price, quarter_end=EXCLUDED.quarter_end;
    """

    # Ensure connection is open before using
    conn = ensure_conn_open(conn)
    with conn.cursor() as cur:
        execute_batch(cur, insert_stmt, data_tuples, page_size=500)
        conn.commit()

    # Per-date stats tracking
    attempted = len(date_prices_df)
    succeeded = date_prices_df["price"].notna().sum()
    success_rate = succeeded / attempted if attempted > 0 else 0.0
    print(f"Date {target_date}: Attempted: {attempted}, Succeeded: {succeeded}, Success Rate: {success_rate:.2%}")

    stats.append({
        "date": target_date,
        "attempted": attempted,
        "succeeded": succeeded,
        "success_rate": success_rate
    })

# Build statistics DataFrame
stats_df = pd.DataFrame(stats)
print("\nStatistics per date:")
print(stats_df)

Fetching for 2019-12-31 ...
Fetched 1/2892: AAPL -> 70.8517
Fetched 2/2892: MSFT -> 149.7332
Fetched 3/2892: META -> 203.8262
Fetched 4/2892: NVDA -> 5.8563
Fetched 5/2892: GOOGL -> 66.4673
Fetched 6/2892: AVGO -> 27.1895
Fetched 7/2892: GOOG -> 66.3533
Fetched 8/2892: AMZN -> 92.392
Fetched 9/2892: TSLA -> 27.8886
Fetched 10/2892: JPM -> 118.1407
Fetched 11/2892: BRK/B -> None
Fetched 12/2892: LLY -> 122.2984
Fetched 13/2892: XOM -> 52.8008
Fetched 14/2892: V -> 180.093
Fetched 15/2892: MA -> 288.6876
Fetched 16/2892: JNJ -> 123.2078
Fetched 17/2892: NFLX -> 32.357
Fetched 18/2892: PLTR -> None
Fetched 19/2892: COST -> 268.7191
Fetched 20/2892: AMD -> 45.86
Fetched 21/2892: BAC -> 30.398
Fetched 22/2892: ABBV -> 69.0318
Fetched 23/2892: WMT -> 36.4032
Fetched 24/2892: HD -> 188.8823
Fetched 25/2892: PG -> 107.5366
Fetched 26/2892: ORCL -> 48.554
Fetched 27/2892: GE -> 54.0989
Fetched 28/2892: CSCO -> 40.0106
Fetched 29/2892: MU -> 52.473
Fetched 30/2892: IBM -> 98.9951
Fetched 31/2892